# 데이터 전처리

마운트 폴더 지정

In [1]:
import sys

In [2]:
sys.path.append(r'C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/src')

데이터 불러오기

In [3]:
import helper_functions
import pet.modeling
import pet.wrapper

In [4]:
import pandas as pd
import numpy as np

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import pet
import os
import numpy as np
import tqdm
import datasets
import pandas as pd

yelp_review_full(2중)

In [6]:
def get_yelp_polarity_data(show_progress_bars=False):

    if not show_progress_bars: datasets.logging.disable_progress_bar()
    yelp_polarity_dataset = datasets.load_dataset('yelp_polarity')
    df_train = pd.DataFrame(yelp_polarity_dataset['train'])
    df_test = pd.DataFrame(yelp_polarity_dataset['test'])
    df_train['label'] = (df_train['label'] + 1).astype(str)
    df_test['label'] = (df_test['label'] + 1).astype(str)
    return df_train, df_test

In [7]:
df_train, df_test = get_yelp_polarity_data(show_progress_bars=False)

2023-08-04 12:30:55,338 - WARNING - builder - Found cached dataset yelp_polarity (C:/Users/user/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61)


In [8]:
pd.value_counts(df_train["label"])

1    280000
2    280000
Name: label, dtype: int64

In [9]:
pd.value_counts(df_test["label"])

2    19000
1    19000
Name: label, dtype: int64

In [14]:
df_train["text"][35]

"Barb's Country Junction gets a 5 star rating for me because it is truly everything that a Pittsburgh restaurant should be.  In addition to being the apple of my nostalgia's eye, it is also the constant thought of my tummy when I'm hungry on the weekends.  \\n\\nBarb's breakfasts cannot be beat, but truly any meal you get there you won't regret.  Everything is made to order, with fair portions, and great prices. My recommendation is a breakfast sandwich on a bagel and a coffee.  There's nothing better than drinking from a cup that could have come from your grandma's kitchen.\\n\\nI know that Gab N Eat is nearby, and Barb's can be a little weird get to with the strange traffic pattern but this really is a gem of Carnegie that you've gotta try!"

전처리 (대문자 소문자 통일, 중복 특수문자 제거)

In [10]:
import re

In [11]:
def cleantext(text):
    text = re.sub(r"[^A-Za-z0-9G-H,.\?!\"']", " ", text)
    text = re.sub(r"([,.\?!\"'])\1+", r"\1", text)
    text = re.sub(r" +", ' ', text)
    return text


In [36]:
df_train['text'] = df_train['text'].apply(lambda x : cleantext(x))

문장 길이 제한

In [37]:
df_train_400 = df_train[df_train['text'].str.len().between(300, 400)].reset_index(drop=True)

train data(문장 제한한 데이터에서 라벨 비율 맞추기)

In [38]:
def sample_data_by_label(dataframe, label_column, label_values, num_samples, random_state=42):
    sampled_data = pd.concat([dataframe[dataframe[label_column] == label].sample(n=num_samples, random_state=random_state)
                             for label in label_values], ignore_index=True)
    return sampled_data

In [39]:
df_train_400_train = sample_data_by_label(df_train_400, 'label', ['1', '2'], 50, random_state=42)

In [40]:
len(df_train_400)

61666

In [41]:
df_train_400_train.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data_2/df_train.csv", index = False)

In [42]:
# unlabeled data  = df_train - df_train_400_train
def merge_and_filter(dataframe1, dataframe2, on_columns, how='left', num_samples=1000):

    merged_df = dataframe1.merge(dataframe2, on=on_columns, how=how, indicator=True)


    filtered_df = merged_df[merged_df['_merge'] == 'left_only']


    filtered_df = filtered_df.drop(columns=['_merge']).reset_index(drop=True)


    num_samples = min(num_samples, filtered_df.shape[0])
    random_indices = np.random.choice(filtered_df.index, num_samples, replace=False)
    filtered_df = filtered_df.loc[random_indices].reset_index(drop=True)

    return filtered_df

In [43]:
# random한 값으로 1100개 뽑기(unlabeled data  = df_train - df_train_400_train)
df_train_400_unlabeled = merge_and_filter(df_train, df_train_400_train, ['label', 'text'], how='left', num_samples=10050)

In [44]:
df_eval_400 = df_train_400_unlabeled[:50]
df_train_400_unlabeled = df_train_400_unlabeled[50:]

In [45]:
len(df_train_400_unlabeled)

10000

In [46]:
df_train_400_unlabeled.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data_2/df_unlabeled.csv", index = False)

In [47]:
df_eval_400.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data_2/df_eval.csv", index = False)

test data

In [48]:
df_test.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data_2/df_test.csv", index = False)